In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_uniform
from torch.autograd import Variable
import numpy as np
#from dataproc import extract_wvs

from math import floor
import random
import sys
import time
import os
from collections import defaultdict

from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, BertModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers.optimization import AdamW, get_linear_schedule_with_warmup


In [2]:
data_dir = "./mimic-iii"
files_dir = "./dump_files"
model_dir = "./models"

In [3]:
from gensim.models import Word2Vec
w2v_model = Word2Vec.load(os.path.join(model_dir, 'w2v-nurse_phys-100-5-3.bin'))

/home/akshara/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
class BaseModel(nn.Module):
    def __init__(self, numClasses, embed_file=None,  dropout=0.2, embed_size=50, vocab_size=500):
        super(BaseModel, self).__init__()
        #torch.manual_seed(1337)
        self.numClasses=numClasses
        self.embed_size = embed_size
        self.dropout = nn.Dropout(p=dropout)
        #make embedding layer
        
        print(vocab_size,embed_size)
        #weights = w2v_model.wv
        #self.embed = nn.Embedding.from_pretrained(weights)
        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        nn.init.xavier_uniform_(self.embed.weight)
        self.embed_size=self.embed.weight.size()[1]

class BERTLATA(BaseModel):
    def __init__(self, numClasses, embed_file, num_filter_maps=256, embed_size=128, dropout=0.2,vocab_size=500):
        super(BERTLATA, self).__init__( numClasses=numClasses, embed_file=embed_file,dropout=dropout, embed_size=embed_size,vocab_size=vocab_size)
        numberofLayers=4
        self.config = BertConfig(vocab_size=self.embed.weight.size()[0], 
        hidden_size=self.embed.weight.size()[1],
        max_position_embeddings=MAX_LEN,
        num_hidden_layers=numberofLayers,
        num_attention_heads=numberofLayers,
        intermediate_size=num_filter_maps,
        output_hidden_states=True)
        
        self.encoder = BertModel(config=self.config)
        self.U = nn.ModuleList([nn.Linear(self.embed.weight.size()[1], self.numClasses) for i in range(numberofLayers+2)])
        self.final=nn.Linear(self.embed.weight.size()[1]*(numberofLayers+2),self.numClasses)

    def forward(self, x,labels=None):
#         print('x', x.shape)
        #x = self.embed(x)
        #print('embed x', x.shape)
        outputs = self.encoder(input_ids=x)
        print()
        attentionOutput=[]
        if(len(outputs)==3):
                attentionOutput=outputs[2]
        elif(len(outputs)==2 ):
                attentionOutput=outputs[1]
        m=[]
        Attentions=[]
        for i, output in enumerate(attentionOutput):
            alpha = F.softmax(self.U[i].weight.matmul(output.transpose(1,2)), dim=2)
            Attentions.append(alpha)
            m1 = alpha.matmul(output)
            m1 = self.dropout(m1)
            m.append(m1)
        alpha = F.softmax(self.U[-1].weight.matmul(outputs[0].transpose(1,2)), dim=2)
        Attentions.append(alpha)
        m1 = alpha.matmul(outputs[0])
        m1 = self.dropout(m1)
        m.append(m1)
        m=torch.cat(m,-1)
        y=self.final.weight.mul(m).sum(dim=2).add(self.final.bias)
        yhat = y
        return yhat, Attentions

class ENCAML(BaseModel):
    def __init__(self,numClasses, embed_file,num_filter_maps=256,embed_size=50, dropout=0.2,vocab_size=500):
        super(ENCAML, self).__init__( numClasses=numClasses, embed_file=embed_file,dropout=dropout, embed_size=embed_size,vocab_size=vocab_size)
        kernel_size=[3,5,7,9]
        self.conv=nn.ModuleList([nn.Conv1d(self.embed_size, num_filter_maps, kernel_size=i, padding=int(floor(i/2))) for i in kernel_size])
        self.U = nn.ModuleList([nn.Linear(num_filter_maps, self.numClasses) for i in range(len(kernel_size))])
        self.final=nn.Linear(num_filter_maps*len(kernel_size),self.numClasses)
        
    def forward(self, x):
        x = self.embed(x)
        x = self.dropout(x)
        x = x.transpose(1, 2)
        m=[]
        Attentions=[]
        for i, conv in enumerate(self.conv):
            x1 = torch.tanh(conv(x).transpose(1,2))
            alpha = F.softmax(self.U[i].weight.matmul(x1.transpose(1,2)), dim=2)
            Attentions.append(alpha)
            m1 = alpha.matmul(x1)
            m1 = self.dropout(m1)
            m.append(m1)
        m=torch.cat(m,-1)
        y=self.final.weight.mul(m).sum(dim=2).add(self.final.bias)
        yhat = y
        return yhat, Attentions

        
#Create a vocabulary file
#criterion=nn.MultiLabelSoftMarginLoss()
#ind2w = {i+2:w for i,w in enumerate(sorted(vocab))}
#ind2w[0]=PAD_TOKEN
#ind2w[1]=UNK_TOKEN
#w2ind = {w:i for i,w in ind2w.items()}
#model=BERTLATA(numClasses,None,64,128,0.2,vocabsize)
#generate batchwise data and target( multi label), in the loop of epochs train model
#outputs, attentions = model(data)
#loss = criterion(outputs, target)


In [5]:
df = pd.read_csv(os.path.join(files_dir, 'nurse_phys.csv'))

In [6]:
tz = BertTokenizer.from_pretrained("bert-base-uncased")
tz.vocab_size

30522

In [7]:
class NotesDataset(Dataset):

    def __init__(self, notes, targets, tokenizer, max_len):
        self.notes = notes
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.notes)

    def __getitem__(self, item):
        review = str(self.notes[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
          review,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
        )
        
        print(encoding.keys())

        return {
          'notes_text': review,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }

In [8]:
RANDOM_SEED = 42
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

In [9]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = NotesDataset(
    notes=df.NOTES.to_numpy(),
    targets=df.iloc[:,2:].to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
    )

    return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
    )

In [10]:
BATCH_SIZE = 10
MAX_LEN = 128

train_data_loader = create_data_loader(df_train, tz, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tz, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tz, MAX_LEN, BATCH_SIZE)

In [11]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model.to(device)
    model = model.train()
    losses = []
    tr_loss = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        
#         print('input', input_ids.shape)
#         print(input_ids)
#         print("target", targets.shape)
#         print(targets)
        outputs, _ = model(input_ids)

        loss = loss_fn(outputs, targets)
        tr_loss += loss.item()

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return tr_loss / n_examples

In [12]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model.to(device)
    model = model.eval()
    tr_loss = 0


    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs, _ = model(
            input_ids=input_ids)

            loss = loss_fn(outputs, targets)
            tr_loss += loss.item()

    return tr_loss / n_examples

In [13]:
def get_predictions(model, data_loader):
    model.to(device)
    model = model.eval()

    predictions = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:
            texts = d["notes_text"]
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs, _ = model(
            input_ids=input_ids)

            probs = F.sigmoid(outputs)

            predictions.extend(preds)
            real_values.extend(targets)

    predictions = torch.stack(predictions).cpu()
    real_values = torch.stack(real_values).cpu()
    return predictions, real_values

In [14]:
numClasses = 6984
EPOCHS = 10
device = 'cuda:1'
model=BERTLATA(numClasses, None, 64, 128, 0.2, 30522)

30522 128


In [15]:
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.MultiLabelSoftMarginLoss()

In [ ]:
%%time

history = defaultdict(list)
best_loss = float('inf')

for epoch in range(EPOCHS):

    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
    )

    print(f'Train loss {train_loss}')

    val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
    )

    print(f'Val loss {val_loss}')
    print()

    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)

    if val_loss < best_loss:
        torch.save(model.state_dict(), 'best_model_state.bin')
        best_loss = val_loss

Epoch 1/1
----------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/akshara/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2073: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to t

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
input torch.Size([1, 128])
tensor([[  101,  3429,  2095,  2214,  3287,  2381,  9820,  3729,  2549,  3728,
         11889,  2349,  7473,  2361,  3591,  3968,  1054, 12205, 20228, 11236,
         18291,  3255,  3138,  2784,  3052, 19340,  1039,  2595,  2099,  3662,
          4800, 14876,  9289,  5443,  7473,  2361,  2363,  3158,  3597,  8292,
          7959,  8197,  4168,  8670,  6593, 20026, 23310, 20784, 20023,  3653,
          2094,  8977,  5643, 11937, 11714,  2361,  2638,  2594,  3054,  1051,
          2475,  2938,  2015,  3968,  2566,  3189,  2872,  1051,  2475,  6948,
          2213,  2588,  5508,  2938,  2015,  2282,  2250, 20228, 11236, 18291,
          3255, 21454,  4728,  3255,  8048,  1060,  8242, 10791, 11265,  4904,
         18981, 18595,  2278,  4530,  7485,  3653,  3540, 13700,  1054, 19857,
         16

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
tensor([[  101,  1052, 12170, 20051, 15321, 22684, 20389,  2100,  7163, 15079,
          7709,  3291,  6412,  7928,  7667, 24978,  2099,  2659,  2927,  5670,
         14925, 14399,  2100,  2358,  6678, 10791,  8242, 10124, 14262,  8820,
          3070, 14931,  2852,  3070,  2895,  3802,  2102,  4654, 28251,  4383,
         22822, 20738, 17153,  9365,  2140,  3255, 10290,  4921,  8840, 20110,
          2953,  1060, 13433, 29454, 10711,  4371,  2213, 17678, 10354, 16515,
          2638, 25606,  3433, 17850, 29454,  2102,  8840, 20110,  2953,  3255,
          2000,  3917,  3085, 13866,  2478,  9174, 28879, 19875,  2000,  3917,
          5844, 10268,  1044,  2475,  2080,  2933,  1051, 16429,  3242,  2572,
         28507,  2618,  2651,  9530,  2102, 17153,  9365,  2140,  3255, 13866,
          2163,  2092,  1059,  2566,  3597,  3401,  2102, 19029,  3046,  2178,
          8700, 20302,  8449,  2594,  9530,  

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
input torch.Size([1, 128])
tensor([[  101,  6522,  2072,  2720,  2095,  2214,  2158,  2381,  1044,  2102,
          2078, 25125,  3742,  7632,  8017, 16298, 29477, 16921,  7175, 22110,
          2278,  4914,  4015, 23025,  2226,  2729,  3322,  3591,  9808,  2232,
          4788,  5582,  1040,  7274,  2361, 22084,  1060,  2420,  1040,  7274,
          2361, 22084,  2195,  2706,  7552, 16007, 28207,  5666,  2147,  6279,
          5994,  8948,  7722,  2195,  3134,  5776,  2179, 24978, 18532,  2072,
          9808,  2232, 19817,  7361,  2733,  3283, 12098,  2546, 13675,  2445,
         17306,  2174,  2445,  2865, 16643, 15460,  3597,  7685,  4015, 15321,
          6305,  6558,  9345,  2140,  3188,  3225,  2002, 19362,  2378, 14181,
          2102,  3968,  3988, 17850, 17531,  10

...

tensor([[0, 0, 0,  ..., 0, 0, 0]], device='cuda:1')
x torch.Size([1, 128])

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
input torch.Size([1, 128])
tensor([[  101, 13866,  4914, 14397,  2226,  1052, 11290, 24501, 18491,  2381,
         16844,  6187, 15253, 23760, 29048,  5099, 23936,  1044, 21833,  6593,
         16940, 17632, 24501, 18491,  7667, 13866,  5777, 23852,  2135,  2305,
          4089, 12098,  3560,  3085,  8048,  1060,  8948,  3154,  2174, 13866,
          5505,  8467,  2335,  2349,  3255,  3571, 21454, 17531, 16922,  1057,
          7361,  2036, 16922, 10507,  2048,  5486,  2847, 16545,  2015,  1060,
         29215,  9333,  2566, 22291,  2136,  3313,  7039, 24582,  2226,  6319,
          2895, 13866,  6628,  7197,  2296,  2847,  2036,  6628, 19340,  2784,
          7200,  2201,  2378,  2445,  1060,  2659,  1057,  7361, 17531,  3433,
          8948,  3961,  3154, 13866,  2583, 19340,  6274,  2505,  1051,  2475,
          2938,  6540

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
input torch.Size([1, 128])
tensor([[  101,  4293,  2095,  2214,  2450,  2381,  1054,  5369, 12248,  3406,
          3593, 27641, 11320, 12207, 23760, 29048, 10886, 17076,  3695, 14728,
          2863, 26261, 13027, 23971,  3593,  2224, 13866, 19077,  2135, 20014,
         19761,  3064,  6540,  4650,  2506,  7620,  3968, 14545,  2607,  8552,
         12436,  2361,  2358,  9331,  2232,  8740, 23446,  8670, 21162, 17577,
         14246,  2278,  7689, 12906,  4607, 24163, 16665,  2140, 21183,  2072,
          4654, 28251,  3370, 23760, 29048, 28378,  7667, 24829,  2361,  2651,
          4921,  6845, 12870,  4135,  2140, 26018,  2721, 21254,  2895,  2979,
          4613, 10577,  9312,  2583,  2707, 13433, 19960,  2015,  2777,  7361,
         13153,  4747, 10514, 14693, 12556, 11460, 13433,  1053,  2154,  9152,
         25

dict_keys(['input_ids', 'attention_mask'])
input torch.Size([1, 128])
tensor([[  101,  6275,  2095,  2214,  2931,  1044,  2595,  1044,  2102,  2078,
         27345,  2594, 15859,  7315,  9808,  2483,  1052,  9033, 21693,  9314,
          5624,  6593, 16940,  1048,  5856,  2497,  2761,  3591,  2902,  4178,
          6065, 14708,  7987,  2497, 18098,  2318,  3110,  5410,  5458,  2170,
          3788,  6411,  4682,  8143,  2098,  7861,  3215,  3369,  2179, 14412,
          4502,  3468,  8187,  3697,  6855, 17531,  2132, 12603, 12603, 10751,
          6540,  3968,  5508,  9808,  2232,  4914,  2966,  2326,  3988, 20228,
          3215,  1999,  2099,  2363,  3197, 10975,  9818,  2015,  7620,  8030,
          9524,  3322,  3030, 16844,  2891,  3597,  7685,  2589,  3041,  2651,
          3662, 27345,  2594, 18845,  6190,  2302,  3161,  9524,  6366,  3461,
         26572,  2361, 18323, 16844,  3461, 26572,  2361,  8292, 24894,  2174,
          2211,  3413,  2312,  8310,  2668, 10975,  3225,  31

dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask'])
input torch.Size([1, 128])
tensor([[  101,  4185,  2095,  2214,  2931,  4914,  7610,  2232, 23852, 23666,
         18629,  3586,  2791,  1040,  7274, 21890, 10440,  2627,  2195,  2086,
          9298,  2290,  1060,  2302,  4335,  3264, 20118, 13320,  2157,  4012,
         24128,  2157,  2364, 13473,  2213, 22953, 12680,  2271,  2482,  4140,
          3593,  5970,  2187,  4942, 20464, 21654,  2706,  3283,  2695,  6728,
          6110,  2157, 11536, 15127, 15321,  6305,  2594, 20118, 13320,  3461,
         21500,  8545,  3726, 22160,  2102, 11320, 11201,  2099, 12475,  9253,
          4958,  2072, 17678, 11253,  4747, 22597,  7610,  2232,  9298,  2290,
          1060,  2487,  7532,  3512,  8153,  8761, 22889,  2063,  1055,  5558,
    

dict_keys(['input_ids', 'attention_mask'])
input torch.Size([1, 128])
tensor([[  101,  6522,  2072,  1042,  2179,  3129,  1054,  3239, 14925, 11714,
         15530,  2483, 13576,  5248,  9741, 18634,  3129,  2170, 29031, 13866,
          4895,  6072, 26029, 12742,  5508,  2579,  2902,  4997,  2132,  1039,
          8560, 14931, 20014, 19761,  3064,  7861, 19009,  8776,  5177,  3570,
          4015,  5729,  1044, 22571,  7856,  7913, 10092,  6583,  3322, 14931,
          2132, 15190,  4997,  1044, 22571,  7856,  7913, 10092,  2659, 13365,
          1044, 22571,  2891,  5302, 13837,  3012,  7667, 18634,  4023,  7391,
          3461, 28022,  2135, 22643,  4076, 10954,  2334, 10057,  2376, 11530,
          2844, 18201, 19340,  1048,  2290,  8310,  3154,  3595,  8496, 17678,
         11253,  4747, 14841,  6494,  3064, 22356, 17531,  2491,  3544,  7327,
          6767, 16930,  2594, 26226,  2361,  2312,  8310,  3154,  3756, 17996,
          4997,  1048,  2947,  2521,  3173, 16731,  5753,  69

In [ ]:
y_pred, y_test = get_predictions(model, test_data_loader)